<a href="https://colab.research.google.com/github/DSGP-Group-1-EAPS/SL-Apparel-Dataset/blob/Ranidu-Gurusinghe/Neural_Network_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Model Code by spliting dataset manually

In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')
# Load the preprocessed data
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DSGP_COURSEWORK/SL Apparel Dataset model/Dataset/preprocessed_data_new.xlsx')
df = df[df['Date'] < '2023-12-01']


Mounted at /content/drive


In [2]:
# Features and target variable
# , 'Encoded Status', 'Encoded Absenteeism Type', 'Encoded Shift', 'DaysWorked'
features = ['Encoded Code', 'NumOfLeaveDays', 'Encoded Reason', 'LeaveYear', 'LeaveMonth']

In [3]:
# Split the data until November 2023 for training and use December 2023 data for testingprint(df['Date'].dtype)
training_data = df[df['Date'] <= '2023-10-31']
testing_data = df[df['Date'] >='2023-11-01']

# Features and target variable for training
X_train = training_data[features]
y_train = training_data['TargetCategory']

# Features and target variable for testing
X_test = testing_data[features]
y_test = testing_data['TargetCategory']

# Standardize the features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the neural network classifier with 3 hidden layers
clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=200, activation='relu',  alpha=0.001, learning_rate='constant', random_state=42)

# Train the neural network on the training data
clf.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_scaled)
y_probs = clf.predict_proba(X_test_scaled)[:, 3]  # Probability of class 'D'

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
accuracy = accuracy*100
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Probability': y_probs  # Probability of class 'D'
})

print(results_df)
print(f"\nAccuracy: {accuracy:.2f}%")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_rep)

# Display all instances where the predicted class is 'D'
predicted_D_instances = results_df[results_df['Predicted'] == 'D']

# Display the employee codes for those instances
employee_codes_predicted_D = testing_data.loc[predicted_D_instances.index, 'Code'].unique()
print("Employee codes with predicted class 'D':")
print(employee_codes_predicted_D)


# Display all instances where the predicted class is 'D' and the prediction is correct
correctly_predicted_D_instances = results_df[(results_df['Predicted'] == 'D') & (results_df['Actual'] == 'D')]

# Display the employee codes for those instances
employee_codes_correctly_predicted_D = testing_data.loc[correctly_predicted_D_instances.index, 'Code'].unique()
print("Employee codes with correctly predicted class 'D':")
print(employee_codes_correctly_predicted_D)


      Actual Predicted  Probability
34040      A         A     0.056029
34041      A         A     0.011546
34042      A         A     0.022571
34043      A         A     0.012758
34044      A         A     0.007403
...      ...       ...          ...
35242      A         A     0.003408
35243      A         A     0.024853
35244      A         A     0.012746
35245      A         A     0.035671
35246      A         A     0.098908

[1175 rows x 3 columns]

Accuracy: 90.81%

Confusion Matrix:
[[1062    0    0   12]
 [  39    0    0    5]
 [  31    0    0    4]
 [  17    0    0    5]]

Classification Report:
              precision    recall  f1-score   support

           A       0.92      0.99      0.96      1074
           B       0.00      0.00      0.00        44
           C       0.00      0.00      0.00        35
           D       0.19      0.23      0.21        22

    accuracy                           0.91      1175
   macro avg       0.28      0.30      0.29      1175
weighted 

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and 

In [4]:
# Assuming 'results_df' is the DataFrame containing prediction results
true_positives_D = results_df[(results_df['Actual'] == 'D') & (results_df['Predicted'] == 'D')]

print("True Positives in Class 'D':")
print(true_positives_D)

True Positives in Class 'D':
      Actual Predicted  Probability
34265      D         D     0.489694
34870      D         D     0.493403
34886      D         D     0.589873
35101      D         D     0.498803
35110      D         D     0.589873


##Hyperparameter tunning

In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the preprocessed data
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DSGP_COURSEWORK/SL Apparel Dataset model/Dataset/preprocessed_data_new.xlsx')
df = df[df['Date'] <= '2023-12-01']
# Features and target variable
features = ['Encoded Code', 'Encoded Reason', 'Encoded Status', 'Encoded Absenteeism Type', 'Encoded Shift', 'DaysWorked', 'DayOfWeek', 'LeaveMonth']

# Split the data until November 2023 for training and use December 2023 data for testing
training_data = df[df['Date'] <= '2023-10-30']

# Features and target variable for training
X_train = training_data[features]
y_train = training_data['TargetCategory']

# Features and target variable for testing
testing_data = df[(df['Date'] > '2023-10-30') & (df['Date'] < '2023-12-01')]
X_test = testing_data[features]
y_test = testing_data['TargetCategory']

# Standardize the features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'hidden_layer_sizes': [(50,), (100, 50), (100, 50, 25), (50, 25, 10)],
    # 'activation': ['relu', 'tanh'],
    # 'solver': ['adam', 'sgd'],
    # 'alpha': [0.0001, 0.001, 0.01],
    # 'learning_rate': ['constant', 'invscaling', 'adaptive'],
    # 'max_iter': [200, 500, 1000],
}

# Initialize the neural network classifier
clf = MLPClassifier(random_state=42)

# Perform grid search with cross-validation
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters
best_clf = MLPClassifier(random_state=42, **best_params)
best_clf.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = best_clf.predict(X_test_scaled)
y_probs = best_clf.predict_proba(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("\nBest Hyperparameters:")
print(best_params)
print(f"\nAccuracy: {accuracy}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_rep)


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the preprocessed data
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DSGP_COURSEWORK/SL Apparel Dataset model/Dataset/preprocessed_data_new.xlsx')
df = df[df['Date'] <= '2023-12-01']


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the preprocessed data
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DSGP_COURSEWORK/SL Apparel Dataset model/Dataset/preprocessed_data_new.xlsx')

df = df[df['Date'] <= '2023-12-01']

# Features and target variable
features = ['Encoded Code', 'Encoded Reason', 'Encoded Status', 'Encoded Absenteeism Type', 'Encoded Shift', 'DaysWorked', 'DayOfWeek', 'LeaveMonth']
X = df[features]
y = df['TargetCategory']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the neural network classifier with 3 hidden layers
clf = MLPClassifier(hidden_layer_sizes=(100,75, 50, 25), max_iter=500, random_state=42)

# Train the model
clf.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Display the results
print(f'Accuracy: {accuracy * 100}%')
print('\nConfusion Matrix:')
print(conf_matrix)
print('\nClassification Report:')
print(classification_rep)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Accuracy: 86.3346680954261%

Confusion Matrix:
[[9914  199   74  691]
 [ 469  770   16  106]
 [ 284   45  649  112]
 [ 362   33   32 3975]]

Classification Report:
              precision    recall  f1-score   support

           A       0.90      0.91      0.91     10878
           B       0.74      0.57      0.64      1361
           C       0.84      0.60      0.70      1090
           D       0.81      0.90      0.86      4402

    accuracy                           0.86     17731
   macro avg       0.82      0.74      0.77     17731
weighted avg       0.86      0.86      0.86     17731

